In [1]:
import csv

annotation_list = ["Tweet Annotations - Feb Translated Annotations.tsv"]
annotation_list.append("Tweet Annotations - Mar Translated Annotations.tsv")
annotation_list.append("Tweet Annotations - Apr Translated Annotations.tsv")

In [131]:
# Transfer the contents of a tsv file to a list for easier access
!pip3 install textblob
from textblob import TextBlob

def tsv_to_list(annotation_file, sentiment=False):
    annotation_list = []
    annotation = open(annotation_file, encoding='utf-8')
    read_tsv = csv.reader(annotation, delimiter="\t") 
    for row in read_tsv:
        if row[2] != '':
            text = TextBlob(row[2])
            if sentiment:
                annotation_list.append([row[0], row[1], row[2], text.sentiment.polarity])
            else:
                annotation_list.append(row)
    print(len(annotation_list))
    return annotation_list

In [3]:
# Get the label for each verse for one annotator
# N: -1, N+NU: -0.5, NU: 0, P+NU: 0.5, P: 1
def get_labels_for(annotation):
    tweets_and_labels = []
    row_num = 0
    for row in annotation:
        if row_num == 0:
            row_num += 1
            continue
        if row[15] != '': # don't use unsure
            row_num += 1
            continue
            #print("not using unsure row", row_num+1)
        if 'UNUSABLE' in row[16].upper() or 'NOT ENOUGH INFO' in row[16].upper():
            row_num += 1
            continue
            #print("unusable row", row_num+1)
        elif row[6].upper() == 'X' and (row[8].upper() != 'X' and row[7].upper() == 'X'): # don't use pos + neg mixed labels
            row_num += 1
            continue
            #print("not using pos+neg row", row_num+1)
        elif row[6].upper() == 'X' and (row[7].upper() != 'X' and row[8].upper() != 'X'): # only pos
            new_row = [annotation[row_num][0], annotation[row_num][1], annotation[row_num][2], annotation[row_num][3], annotation[row_num][4], annotation[row_num][5], 1]
            tweets_and_labels.append(new_row)
        #elif row[6].upper() == 'X' and (row[8].upper() == 'X' and row[7].upper() != 'X'): # pos + neu
        #    new_row = [annotation[row_num][0], annotation[row_num][1], annotation[row_num][2], annotation[row_num][3], annotation[row_num][4], annotation[row_num][5], 3]
        #    tweets_and_labels.append(new_row)
        elif row[8].upper() == 'X' and (row[6].upper() != 'X' and row[7].upper() != 'X'): # only neu 
            new_row = [annotation[row_num][0], annotation[row_num][1], annotation[row_num][2], annotation[row_num][3], annotation[row_num][4], annotation[row_num][5], 0]
            tweets_and_labels.append(new_row)
        elif row[7].upper() == 'X' and (row[8].upper() != 'X' and row[6].upper() != 'X'): # only neg
            new_row = [annotation[row_num][0], annotation[row_num][1], annotation[row_num][2], annotation[row_num][3], annotation[row_num][4], annotation[row_num][5], -1]
            tweets_and_labels.append(new_row)
        #elif row[8].upper() == 'X' and (row[7].upper() == 'X' and row[6].upper() != 'X'): # neg + neu
        #    new_row = [annotation[row_num][0], annotation[row_num][1], annotation[row_num][2], annotation[row_num][3], annotation[row_num][4], annotation[row_num][5], 2]
        #    tweets_and_labels.append(new_row) 
        #else:
        #    print("outlier row:", row_num+1)
        row_num += 1
    return tweets_and_labels

In [77]:
feb_list = tsv_to_list(annotation_list[0])
feb_labels = get_labels_for(feb_list)
mar_list = tsv_to_list(annotation_list[1])
mar_labels = get_labels_for(mar_list)
apr_list = tsv_to_list(annotation_list[2])
apr_labels = get_labels_for(apr_list)

all_labels = feb_labels + mar_labels + apr_labels
#print(all_labels.count(-1), all_labels.count(0), all_labels.count(1))

import pandas as pd
data = pd.DataFrame(all_labels, columns=['tweetID', 'tweetTime', 'tweetText', 'tweetLang', 'tweetCoordinates', 'tweetPlace', 'tweetSentiment'])
data.head()

tweets_labels = pd.DataFrame(index=None, columns=['tweetText', 'tweetSentiment'])
positive_count = 0
neutral_count = 0

for i in range(len(data.index)):
    if data.at[i, 'tweetSentiment'] == 1:
        positive_count += 1
    if data.at[i, 'tweetSentiment'] == 0:
        neutral_count += 1

print(positive_count, neutral_count)

neg_count = 0
for i in range(len(data.index)):
    if neg_count < neutral_count:
        if data.at[i, 'tweetSentiment'] == -1:
            neg_count += 1
            tweets_labels.loc[len(tweets_labels.index)] = [data.at[i, 'tweetText'], data.at[i, 'tweetSentiment']]
        else:
            tweets_labels.loc[len(tweets_labels.index)] = [data.at[i, 'tweetText'], data.at[i, 'tweetSentiment']]
            
tweets_labels.info()
#test_data = pd.DataFrame(apr_labels, columns=['tweetID', 'tweetTime', 'tweetText', 'tweetLang', 'tweetCoordinates', 'tweetPlace', 'tweetSentiment'])

769 925
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1371 entries, 0 to 1370
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tweetText       1371 non-null   object
 1   tweetSentiment  1371 non-null   object
dtypes: object(2)
memory usage: 32.1+ KB


In [68]:
#!pip3 install nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import PunktSentenceTokenizer
from nltk import NaiveBayesClassifier
from nltk.classify import accuracy

In [78]:
# Get all the words from the data
import random

text = []
for tweet in tweets_labels['tweetText']:
    text.append(tweet.split())
print(len(text))

tweets_and_labels = []
tweet_count = 0
for label in tweets_labels['tweetSentiment']:
    if tweet_count < len(tweets_labels['tweetSentiment']):
        tweets_and_labels.append((text[tweet_count], label))
        tweet_count += 1
print(len(tweets_and_labels))    

0       I love to look at people's opinions about the ...
1       The latest The AmericanAwaken ing Daily PM!  T...
2       #Macau casinos will be closed for half a month...
3       Social media conspiracies blame coronavirus on...
4       Coronavirus, official: patient zero has nothin...
                              ...                        
1366    Coachella music festival postponed over corona...
1367    If later one messes with the gypsies of the 30...
1368    That's the whole point! Here is his moment to ...
1369    Sumo: March tournament to be held behind close...
1370    @guardian No, I'm pretty sure it's just becaus...
Name: tweetText, Length: 1371, dtype: object
1371
1371


In [80]:
random.shuffle(tweets_and_labels)

In [81]:
all_words = []

for sent in text:
    for w in sent:
        all_words.append(w.lower())

all_words = FreqDist(all_words)

word_features = list(all_words.keys())[:3000]
print(word_features[:20])

['i', 'love', 'to', 'look', 'at', "people's", 'opinions', 'about', 'the', 'coronavirus', 'because', 'can', 'find', 'both', 'memes', 'and', 'people', 'who', 'believe', 'that']


In [82]:
def find_features(document):
    words = set(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)
        #print(w, ":", features[w])

    return features

In [84]:
featuresets = [(find_features(tweet), category) for (tweet, category) in tweets_and_labels]

In [86]:
cutoff = round(len(tweets_and_labels)*0.9) - 1
training_set = featuresets[:cutoff]
testing_set = featuresets[cutoff:]
print(cutoff)

1233


In [87]:
classifier = NaiveBayesClassifier.train(training_set)

In [88]:
print("Classifier accuracy percent:",(accuracy(classifier, testing_set))*100)

Classifier accuracy percent: 59.42028985507246


In [89]:
tweet = 'Just 9 cases, 7 of them linked, from 60,000 tests! Outstanding! NSW absolutely smashing it  Thanks so much to our brilliant contact tracers'
classifier.classify(find_features(tweet.split()))

-1

In [91]:
classifier.show_most_informative_features(10)

Most Informative Features
                negative = True                1 : -1     =     16.4 : 1.0
                 contain = True                1 : -1     =     12.8 : 1.0
             temperature = True                1 : -1     =     12.8 : 1.0
                   fight = True                1 : -1     =     11.7 : 1.0
                    2019 = True                0 : -1     =      9.9 : 1.0
                   areas = True                1 : -1     =      9.1 : 1.0
                    care = True                1 : -1     =      9.1 : 1.0
                carrying = True                1 : -1     =      9.1 : 1.0
            coronavirus" = True                1 : -1     =      9.1 : 1.0
                  demand = True                1 : -1     =      9.1 : 1.0


In [122]:
nov_list = tsv_to_list('Tweet Annotations - Nov Translated.tsv')
results = pd.DataFrame(index=None, columns=['text', 'sentiment'])
count = 0
for row in nov_list:
    tweetText = row[2]
    sentiment = classifier.classify(find_features(tweetText.split()))
    results.loc[len(results.index)] = [tweetText, sentiment]
    count += 1
    if count == 100:
        break
results.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       100 non-null    object
 1   sentiment  100 non-null    object
dtypes: object(2)
memory usage: 2.3+ KB


In [124]:
results.sample(n=50)

,text,sentiment
37,Election: Biden pledges to work for unity and ...,-1
44,"Sindh reports 1,276 new cases, highest daily i...",-1
47,Despite the challenges we face because of the ...,-1
14,Wuhan's atmosphere is inundated with #WuhanVir...,-1
53,1of2. This like what happened 2 me with our He...,-1
82,Why does the early Nov COVID map mirror the el...,0
75,Magnesium plays an extremely important role in...,-1
17,"if not enough coronavirus, burning and all the...",-1
50,Media is unhinged knowing nothing can stop wha...,-1
42,@rtenews Such rubbish. People can't be punish ...,-1


In [132]:
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec', 'Jan2021']
all_tweets = []
for month in months:
    filename = 'Tweet Annotations - '+month+' Translated.tsv'
    temp_list = tsv_to_list(filename, True)
    all_tweets += temp_list

full_df = pd.DataFrame(all_tweets, columns=['id', 'date', 'text', 'sentiment'])
full_df.head()

5375
5330
5332
5351
4898
4944
4922
4903
4977


,id,date,text,sentiment
0,tweetID,tweetTime,tweetText,0.0
1,1267113249692057606,2020-05-31 15:18:35,@vogon Good Morning Amerikkka Expect more ...,0.6
2,1266879605216030721,2020-05-30 23:50:10,"The # COVID19 #VaiPassar, CAMPOS DO JORDAO / S...",0.0
3,1258009083577274368,2020-05-06 12:21:52,This is how it was in audiences to Win Sports ...,0.4
4,1261087171273994241,2020-05-15 0:13:06,Nurses are not getting the protections they de...,-0.6
